<a href="https://colab.research.google.com/github/Kotzly/DS4H_Course/blob/main/notebooks/colab_sinasc_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install simpledbf

  Created wheel for simpledbf: filename=simpledbf-0.2.6-cp37-none-any.whl size=13793 sha256=cf79f901e9e387570d3ec495452a436147c06165e34635b5703443c4870e4f8c
  Stored in directory: /root/.cache/pip/wheels/72/dd/df/cdfb970a508ef9750ebb4b6e035a3410c3d62b5a6a91d2aafc
Successfully built simpledbf


In [7]:
import shutil
import urllib.request as request
from contextlib import closing
import zipfile
import shutil
import sys
import os
from pathlib import Path
import pandas as pd
from simpledbf import Dbf5
import platform

def download(url, save_filepath):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(save_filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)

def download_ftp(url, savepath):
    with closing(request.urlopen(url)) as r:
        with open(savepath, 'wb') as f:
            shutil.copyfileobj(r, f)

root = Path(".")

In [8]:
dbc_save_folder = root / "SINASC_DBC"
dbf_save_folder = root / "SINASC_DBF"
csv_save_folder = root / "SINASC_CSV"

dbc_save_folder.mkdir(exist_ok=True)
dbf_save_folder.mkdir(exist_ok=True)
csv_save_folder.mkdir(exist_ok=True)


In [9]:
SINASC_URLS = [
    f"ftp://ftp.datasus.gov.br/dissemin/publicos/SINASC/1996_/Dados/DNRES/DNSP{year}.dbc"
    for year in range(2010, 2019 + 1)
]



for url in SINASC_URLS:
    print("Downloading", url.split("/")[-1])
    save_path = dbc_save_folder / url.split("/")[-1]
    download_ftp(url, save_path)

In [10]:
%load_ext rpy2.ipython

In [11]:
%%R
# Using read.dbc package
# https://pt.linkedin.com/pulse/datasus-conhe%C3%A7a-nova-ferramenta-para-ler-arquivos-dbc-petruzalek

install.packages("read.dbc")
library("read.dbc")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/read.dbc_1.0.5.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 20988 bytes (20 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write

In [12]:
!ls SINASC_CSV

In [13]:
%%R
files = list.files("SINASC_DBC")
for (file in files){
    
    print(file)

    filename = strsplit(file, ".dbc")[1]
    
    dbc_filepath = paste("./SINASC_DBC/", file, sep="")

    csv_filepath = paste("./SINASC_CSV/", filename, ".csv", sep="")
    
    df <- read.dbc(dbc_filepath)

    write.csv(df, csv_filepath)
}

[1] "DNSP2010.dbc"
[1] "DNSP2011.dbc"
[1] "DNSP2012.dbc"
[1] "DNSP2013.dbc"
[1] "DNSP2014.dbc"
[1] "DNSP2015.dbc"
[1] "DNSP2016.dbc"
[1] "DNSP2017.dbc"
[1] "DNSP2018.dbc"
[1] "DNSP2019.dbc"


In [14]:
union_df = None
dfs = []
cols = ["DTNASC", "QTDFILVIVO", "SEXO", "IDADEMAE", "PESO", "GRAVIDEZ", "CONSULTAS", "RACACOR", "CODMUNNASC", "ESTCIVMAE", "ESCMAE", "PARTO", "IDANOMAL"]
union_csv_filename = "union.csv"
for csv_filepath in csv_save_folder.glob("*.csv"):
    if csv_filepath.name == union_csv_filename:
        continue
    df = pd.read_csv(csv_filepath, usecols=cols)
    print(csv_filepath, len(df.columns))
    sorted_cols = list(sorted(df.columns))
    df = df[cols]
    dfs += [df]
    if union_df is None:
        union_df = df
    else:
        union_df = pd.concat([union_df, df])
union_df.to_csv(csv_save_folder / union_csv_filename)

SINASC_CSV/DNSP2018.csv 13
SINASC_CSV/DNSP2017.csv 13
SINASC_CSV/DNSP2012.csv 13
SINASC_CSV/DNSP2010.csv 13
SINASC_CSV/DNSP2011.csv 13
SINASC_CSV/DNSP2019.csv 13
SINASC_CSV/DNSP2014.csv 13
SINASC_CSV/DNSP2016.csv 13
SINASC_CSV/DNSP2015.csv 13
SINASC_CSV/DNSP2013.csv 13


In [15]:
import pandas as pd

pd.read_csv(csv_save_folder / "union.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,DTNASC,QTDFILVIVO,SEXO,IDADEMAE,PESO,GRAVIDEZ,CONSULTAS,RACACOR,CODMUNNASC,ESTCIVMAE,ESCMAE,PARTO,IDANOMAL
0,0,6012018,0.0,1,32.0,4290.0,1.0,4.0,4.0,140010,2.0,4.0,2.0,2.0
1,1,22052018,1.0,2,23.0,2794.0,1.0,4.0,4.0,150130,5.0,4.0,2.0,2.0
2,2,14042018,0.0,1,22.0,2460.0,1.0,3.0,1.0,150140,5.0,4.0,2.0,2.0
3,3,12032018,1.0,2,23.0,3055.0,1.0,3.0,4.0,150170,1.0,4.0,1.0,2.0
4,4,7042018,3.0,2,33.0,2936.0,1.0,4.0,4.0,170950,1.0,5.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6101363,610891,26122013,4.0,1,42.0,3340.0,1.0,3.0,4.0,231410,1.0,3.0,1.0,2.0
6101364,610892,4122013,3.0,1,31.0,3290.0,1.0,9.0,NaN,230440,2.0,2.0,2.0,2.0
6101365,610893,30122013,0.0,1,25.0,3120.0,1.0,3.0,1.0,420820,5.0,4.0,1.0,2.0
6101366,610894,5102013,4.0,2,28.0,2660.0,1.0,2.0,1.0,353060,1.0,4.0,2.0,2.0


In [51]:
df = pd.read_csv(csv_save_folder / "union.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
cities = {
  3502101: 'Andradina',
  3502804: 'Araãatuba',
  3505708: 'Barueri',
  3514403: 'Dracena',
  3517406: 'Guaãra',
  3524808: 'Jales',
  3546801: 'Santa Isabel',
  3548500: 'Santos',
  3548807: 'São Caetano do Sul',
  3549805: 'São Josã do Rio Preto',
  3550308: 'São Paulo'
}
cities = {(cod // 10): name for cod, name in cities.items()}
cities

{350210: 'Andradina',
 350280: 'Araãatuba',
 350570: 'Barueri',
 351440: 'Dracena',
 351740: 'Guaãra',
 352480: 'Jales',
 354680: 'Santa Isabel',
 354850: 'Santos',
 354880: 'São Caetano do Sul',
 354980: 'São Josã do Rio Preto',
 355030: 'São Paulo'}

In [67]:
def parse_day(d):
  d = str(d)
  day, month, year = int(d[-8:-6]), int(d[-6:-4]), int(d[:-4])
  return day
def parse_month(d):
  d = str(d)
  day, month, year = int(d[-8:-6]), int(d[-6:-4]), int(d[:-4])
  return month
def parse_year(d):
  d = str(d)
  day, month, year = int(d[-8:-6]), int(d[-6:-4]), int(d[-4:])
  if year < 2000:
    year += 2000
  return year

df = df[df.CODMUNNASC.apply(lambda x: x in cities.keys())]

df["YEAR"], df["MONTH"], df["DAY"] = 0, 0, 0
df["DAY"] = df.DTNASC.apply(parse_day)
df["MONTH"] = df.DTNASC.apply(parse_month)
df["YEAR"] = df.DTNASC.apply(parse_year)
df["CITYNAME"] = df.CODMUNNASC.apply(lambda x: cities[x])

In [68]:
df

,Unnamed: 0,DTNASC,QTDFILVIVO,SEXO,IDADEMAE,PESO,GRAVIDEZ,CONSULTAS,RACACOR,CODMUNNASC,ESTCIVMAE,ESCMAE,PARTO,IDANOMAL,YEAR,MONTH,DAY,CITYNAME
3435,3435,1012018,1.0,1,37.0,3740.0,1.0,4.0,1.0,350210,2.0,5.0,2.0,9.0,2018,1,1,Andradina
3436,3436,1012018,0.0,1,30.0,3330.0,1.0,4.0,1.0,350210,2.0,4.0,2.0,9.0,2018,1,1,Andradina
3437,3437,2012018,1.0,1,36.0,3320.0,1.0,4.0,4.0,350210,1.0,4.0,2.0,9.0,2018,1,2,Andradina
3438,3438,2012018,0.0,1,22.0,2885.0,1.0,4.0,1.0,350210,5.0,4.0,2.0,9.0,2018,1,2,Andradina
3439,3439,3012018,0.0,1,22.0,3280.0,1.0,4.0,1.0,350210,1.0,4.0,2.0,9.0,2018,1,3,Andradina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6100437,609965,31122013,2.0,1,24.0,2310.0,1.0,2.0,4.0,355030,1.0,3.0,1.0,2.0,2013,12,31,São Paulo
6101239,610767,20122013,9.0,2,38.0,2055.0,1.0,3.0,4.0,354980,1.0,4.0,1.0,2.0,2013,12,20,São Josã do Rio Preto
6101240,610768,14112013,0.0,2,17.0,2895.0,1.0,4.0,4.0,354980,1.0,3.0,1.0,2.0,2013,11,14,São Josã do Rio Preto
6101243,610771,29042013,99.0,2,29.0,2305.0,1.0,9.0,NaN,355030,9.0,9.0,2.0,9.0,2013,4,29,São Paulo


In [98]:
pd.set_option('display.max_rows', None)

count_df = df.groupby(["YEAR", "MONTH", "CITYNAME", "CODMUNNASC"]).count()[["DTNASC"]]

DTNASC
YEAR MONTH CITYNAME              CODMUNNASC        
2010 1     Andradina             350210          86
           Araãatuba             350280         217
           Barueri               350570         591
           Dracena               351440          47
           Guaãra                351740          32
           Jales                 352480          80
           Santa Isabel          354680          75
           Santos                354850         993
           São Caetano do Sul    354880          91
           São Josã do Rio Preto 354980         682
           São Paulo             355030       16510
     2     Andradina             350210          77
           Araãatuba             350280         232
           Barueri               350570         537
           Dracena               351440          69
           Guaãra                351740          35
           Jales                 352480          79
           Santa Isabel          354680          82
           Santos                354850         976
           São Caetano do Sul    354880          93
           São Josã do Rio Preto 354980         662
           São Paulo             355030       15951
     3     Andradina             350210          87
           Araãatuba             350280         266
           Barueri               350570         616
           Dracena               351440          71
           Guaãra                351740          49
           Jales                 352480          82
           Santa Isabel          354680          89
           Santos                354850        1178
           São Caetano do Sul    354880         100
           São Josã do Rio Preto 354980         823
           São Paulo             355030       18063
     4     Andradina             350210          91
           Araãatuba             350280         262
           Barueri               350570         559
           Dracena               351440          59
           Guaãra                351740          40
           Jales                 352480          80
           Santa Isabel          354680          69
           Santos                354850        1007
           São Caetano do Sul    354880          87
           São Josã do Rio Preto 354980         709
           São Paulo             355030       16150
     5     Andradina             350210          70
           Araãatuba             350280         220
           Barueri               350570         580
           Dracena               351440          49
           Guaãra                351740          33
           Jales                 352480          98
           Santa Isabel          354680          57
           Santos                354850        1074
           São Caetano do Sul    354880          90
           São Josã do Rio Preto 354980         695
           São Paulo             355030       16717
     6     Andradina             350210          71
           Araãatuba             350280         235
           Barueri               350570         533
           Dracena               351440          39
           Guaãra                351740          48
           Jales                 352480          80
           Santa Isabel          354680          68
           Santos                354850        1019
           São Caetano do Sul    354880         127
           São Josã do Rio Preto 354980         598
           São Paulo             355030       15977
     7     Andradina             350210          65
           Araãatuba             350280         218
           Barueri               350570         530
           Dracena               351440          56
           Guaãra                351740          35
           Jales                 352480          85
           Santa Isabel          354680          72
           Santos                354850        1052
           São Caetano do Sul    354880         136
           São Josã do Rio Preto 354980  